In [2]:
import pandas as pd
from scipy.interpolate import interp1d
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import price_data as price
import ccxt
import time
import chart


In [3]:
ftx = ccxt.ftx({
    'apiKey': 'mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',
    'secret': 'oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B',
    'enableRateLimit': True,
})
cex=ccxt.cex({
    'apiKey': '5ZxUkOA6EK18l15M5MVkAyoLk',
    'secret': 'qeME2eoB1MPuaLz6z51zUiWlD5A',
    'enableRateLimit': True,
})

In [4]:
results_df = pd.read_csv('data/results.csv')
results_df.iloc[results_df['success_rate'].idxmax()]

Unnamed: 0      368.0000
ema              28.0000
channel          18.0000
success_rate      0.9375
Name: 368, dtype: float64

In [5]:
float(next(filter(lambda x: x['coin']=='USD',ftx.fetch_balance()['info']['result']))['total'])

64.44664058

In [21]:
abs(np.log10(float(next(filter(lambda x:x['symbol']=='ETH/USD',ftx.fetch_markets()))['precision']['amount'])))

3.0

In [6]:
next(filter(lambda x: x['future']=='ETH-PERP',ftx.fetch_positions()))

{'future': 'ETH-PERP',
 'size': 0.0,
 'side': 'buy',
 'netSize': 0.0,
 'longOrderSize': 0.0,
 'shortOrderSize': 0.0,
 'cost': 0.0,
 'entryPrice': None,
 'unrealizedPnl': 0.0,
 'realizedPnl': 8.07911562,
 'initialMarginRequirement': 1.0,
 'maintenanceMarginRequirement': 0.03,
 'openSize': 0.0,
 'collateralUsed': 0.0,
 'estimatedLiquidationPrice': None}

In [ ]:
int(time.time())/3600 == int(time.time())//3600

False

In [ ]:
daily=price.get_price_data('1d',symbol='ETH/USD')
hourly=price.get_price_data('1h',symbol='ETH/USD')
# gradient=chart.get_gradient(chart.get_ema(daily,28))

In [9]:
price.get_current_price('ETH/USD')

1945.1

In [4]:
cex.fetch_balance()

AuthenticationError: requires `uid`